# Deploy a custom classification model from Watson NLP as a Python function

This notebook shows the steps needed to create and deploy a Python function that classifies incoming documents. The classification is done by a custom model `classification_model_rt24_1`, which is part of this sample project. The example creates an _online deployment_ of the Python function. This way, the classification can be invoked via a web service in applications.

## Prerequisites 

Before running this notebook:
1. Create a new deployment space, or identify an existing deployment space to use for this example
2. Open the `Assets` tab of this sample project
3. Click the three-dot-menu for `classification_model_rt24_1` and select `Promote to space`
4. As target space, use the deployment space from step 1

## Initialize the Watson Machine Learning client

In [ ]:
CPD_URL = '<CPD PLATFORM URL>'

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [ ]:
wml_credentials = {
    "url": CPD_URL,
    "token": wslib.auth.get_current_token(),
    "instance_id": "wml_local",
    "version" : "5.0"
}

In [ ]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
client.spaces.list()

### Add the `ID` of the space to use

In [ ]:
client.set.default_space('<YOUR SPACE>')

## Create a Python function that classifies documents

The model that is used here is the SVM model that is created in the example notebook `Complaint Classification`, section _Train a SVM classification model with Watson NLP_. The input data for _training_ the model was prepared in two steps:
1. Syntax Analysis
2. Creation of USE embeddings

Hence, for _applying_ the model on new data, the input data has to be transformed in the same way.

In [ ]:
def detect_category():
    import watson_nlp
    
    # Syntax Model
    syntax_model = watson_nlp.load('syntax_izumo_en_stock')
    # USE Embedding Model
    use_model = watson_nlp.load('embedding_use_en_stock')
    
    #custom classification model
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()
    
    svm_model = watson_nlp.load(wslib.load_data('classification_model_rt24_1'))
      
    
    def score(input):
        scoring_prediction_out = []
        for input_data_row in input["input_data"][0]["values"]: 
            scoring_prediction_row = []
            for input_data in input_data_row:
                # run syntax model first
                syntax_result = syntax_model.run(input_data)
                # run custom model on top of syntax result
                svm_preds = svm_model.run(use_model.run(syntax_result, doc_embed_style='raw_text'))
                
                scoring_prediction_row.append(svm_preds.to_dict())
                
            scoring_prediction_out.append(scoring_prediction_row)
            
        # Score using the pre-defined model
        scoring_response = {
            'predictions': [{'fields': ['nlp_prediction'], 
                             'values': scoring_prediction_out
                            }]
        }

        return scoring_response
    
    return score

## Save the Python function to the Model repository

In [ ]:
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-24.1-py3.11")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'classification_function',
    client.repository.FunctionMetaNames.DESCRIPTION: 'classification_function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


classification_function_details = client.repository.store_function(meta_props=meta_data, 
                                                                   function=detect_category)

## Create an Online Deployment

In [ ]:
classification_function_uid = client.repository.get_function_uid(classification_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "nlp-classification-deploy",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "nlp-classification-deploy",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

classification_deployment_details = client.deployments.create(classification_function_uid, 
                                                              meta_props=meta_props)
classification_deployment_id = client.deployments.get_uid(classification_deployment_details)

## Classify sample complaints

### Download the complaint data

In [ ]:
import pandas as pd
url = "https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max=2021-03-30&date_received_min=2021-02-28&field=all&format=csv&has_narrative=true&no_aggs=true&size=18102"

complaint_df = pd.read_csv(url)
text_col = 'Consumer complaint narrative'

complaint_df.head(2)

### Collect some sample complaints as input

In [ ]:
complaints = complaint_df['Consumer complaint narrative'].values.tolist()

inputData = []
for i in range(1,30):
    inputData.append([complaints[i]])

In [ ]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{  
        'fields': ["Text"],
        'values': inputData
   }]
}

### Predict the complaint category

In [ ]:
predictions = client.deployments.score(classification_deployment_id, scoring_payload)
predictions

## Clean up: Delete the deployment and the function

In [ ]:
client.deployments.delete(classification_deployment_id)

In [ ]:
client.repository.delete(classification_function_uid)